In [2]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [70]:
chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,6,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

J=1

htemp=(KP(s0,sx,sx,s0)+KP(s0,sz,sz,s0))+\
    J*(KP(sx,sx,s0,s0)+KP(sz,sz,s0,s0)+KP(s0,s0,sx,sx)+KP(s0,s0,sz,sz))/2
# htemp=KP(sx,sx)+J*(KP(sz,s0)+KP(s0,sz))/2
# htemp=KP(sx,sx)+KP(sy,sy)+KP(sz,sz)

d=int(np.sqrt(htemp.shape[0]))
Ainit=expm(-0.002*htemp).reshape(d,d,d,d)
A0=Asplit(Ainit,chi)
for i in range(8):
    A0=coarsen(A0,chi)

# na=np.newaxis
# s=np.array([1,-1])
# Tc=2/np.log(1+np.sqrt(2))
# T=1*Tc
# A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
#                 +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
# A0/=np.linalg.norm(A0)

vertical truncation err: 0.6242497505002912
horizontal truncation err: 0.0
Aout shape: (6, 6, 6, 6)
Coarsen truncation error: 3.16407e-15
Coarsen truncation error: 3.16387e-15
Coarsen truncation error: 2.99658e-15
Coarsen truncation error: 3.15982e-15
Coarsen truncation error: 3.1469e-15
Coarsen truncation error: 3.25872e-15
Coarsen truncation error: 3.05294e-15
Coarsen truncation error: 2.98017e-15


In [71]:
RGsteps=10
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm=doTNR(Aold,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm
    print('A.shape= ',A.shape)
    print()

RG step:  0
iter=0, U.Uenv=1
iter=100, U.Uenv=1
error in U, vl, vr optimization: 0
error in yl, yr, D optimization: 2.93331e-15
error in w optimization: 0.219704
iter=0, u.uenv=0.932864, v.venv=0.932864
iter=1000, u.uenv=0.932864, v.venv=0.932864
Adiff 0.366432
A.shape=  (6, 6, 6, 6)

RG step:  1
iter=0, U.Uenv=1
iter=100, U.Uenv=1
error in U, vl, vr optimization: -1.77773e-16
error in yl, yr, D optimization: 2.99193e-15
error in w optimization: 0.0371804
iter=0, u.uenv=0.97047, v.venv=0.97047
iter=1000, u.uenv=0.97047, v.venv=0.97047
Adiff 0.243023
A.shape=  (6, 6, 6, 6)

RG step:  2
iter=0, U.Uenv=1
iter=100, U.Uenv=1
error in U, vl, vr optimization: -7.58032e-16
error in yl, yr, D optimization: 3.00591e-15
error in w optimization: 0.000619077
iter=0, u.uenv=0.999331, v.venv=0.999331
iter=1000, u.uenv=0.999331, v.venv=0.999331
Adiff 0.0365679
A.shape=  (6, 6, 6, 6)

RG step:  3
iter=0, U.Uenv=1
iter=100, U.Uenv=1
error in U, vl, vr optimization: 1.91099e-06
error in yl, yr, D optimiz

In [72]:
i=10
A=A_list[i]
import scipy

In [73]:
T=np.einsum(A,[2,1,2,0])
# ew,_=np.linalg.eigh(T)
ew = scipy.sparse.linalg.eigsh(T, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[5]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([-0., nan, nan, nan, nan, nan])

In [74]:
T=A[0,:,0,:]
ew,_=np.linalg.eigh(T)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[4]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


array([-0., nan, nan, nan, nan, nan])

In [75]:
T2=np.einsum(A,[4,2,5,0],A,[5,3,4,1]).reshape(chiw**2,chiw**2)
ew,_=np.linalg.eigh(T2)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[5]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


array([-0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [76]:
T3=np.einsum(A,[10,3,8,0],A,[8,4,9,1],A,[9,5,10,2]).reshape(chiw**3,chiw**3)
ew = scipy.sparse.linalg.eigsh(T3, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[6]-el[0]))[:100]

array([-0.        ,  0.98533899,  0.99278887,  0.99430289,  0.99713318,
        0.99904217,  1.        ,  1.0023255 ,  1.0032207 ,  1.00385318,
        1.00403879,  1.00444692,  1.00450745,  1.00474447,  1.00479912,
        1.00481575,  1.00492926,  1.00501821,  1.00506891,  1.0056507 ,
        1.00569602,  1.00589928,  1.00614889,  1.00639831,  1.00670451,
        1.00707821,  1.00761251,  1.00845214,  1.00966963,  1.00972575,
        1.0097769 ,  1.00982887,  1.01020339,  1.01051362,  1.01051619,
        1.01114142,  1.01132681,  1.01321101,  1.01323903,  1.01327087,
        1.01472869,  1.01635406,  1.01836712,  1.01837982,  1.01862126,
        1.0203181 ,  1.02239474,  1.02555559,  1.03413122,  1.03499584,
        1.03598029,  1.05097496,  1.07560162,  1.04438134,  1.03933278,
        1.03105325,  1.02534444,  1.02144426,  1.02106603,  1.01883488,
        1.01865191,  1.01658911,  1.01608625,  1.01341637,  1.0121233 ,
        1.01085562,  1.01044105,  1.00889795,  1.00805731,  1.00

In [84]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew,_ = np.linalg.eig(T4)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el)[:100]

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([-0., inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf])

In [86]:
ew

array([1.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])

In [82]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew = scipy.sparse.linalg.eigsh(T4, k=100, which='LM', return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

array([-0.        ,  0.98177172,  0.99669649,  0.99803865,  0.9986423 ,
        0.9987946 ,  0.99896682,  0.99909685,  0.99949844,  1.        ,
        1.00009002,  1.00029668,  1.00085156,  1.00109714,  1.00207088,
        1.00213751,  1.00233087,  1.00297601,  1.00313051,  1.0032219 ,
        1.00333175,  1.00335376,  1.00400226,  1.00411341,  1.00437936,
        1.00460477,  1.00546215,  1.00629849,  1.00655387,  1.00700243,
        1.00739674,  1.00915915,  1.00947787,  1.00973638,  1.01030318,
        1.01297456,  1.01440127,  1.01516865,  1.01517105,  1.01736849,
        1.01741433,  1.01760444,  1.01828501,  1.01842282,  1.02002835,
        1.02283608,  1.0440878 ,  1.03730653,  1.03221008,  1.0319128 ,
        1.02529835,  1.02477686,  1.02182697,  1.02069047,  1.02037569,
        1.01740282,  1.01486694,  1.01168791,  1.01158466,  1.01093676,
        1.01086619,  1.00971888,  1.00907916,  1.0081681 ,  1.0081387 ,
        1.0080972 ,  1.00799428,  1.00768553,  1.00715563,  1.00

In [79]:
T4reduced=np.einsum(A,[11,0,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,3,11,3]).reshape(chiw**2,chiw**2)
ew = scipy.sparse.linalg.eigsh(T4reduced, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


array([-0., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [38]:
MOq.shape

(4, 4, 4, 4, 4, 4, 4, 4)

In [37]:
#eval mag
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz
A0=Asplit(Ainit,chi)
Az=Az_(Ainit,chi)
MOq=MOq_(A0,Az)

eval_op(MOq,tensor_list)  

vertical truncation err: 0.0
horizontal truncation err: 0.0
Aout shape: (4, 4, 4, 4)


ValueError: Size of label 'l' for operand 1 (4) does not match previous terms (6).

In [28]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [24]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]

gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)

chiytemp=min(chiy,Aold.shape[0])
chiwtemp=min(chiw,Aold.shape[1])
def scaleop(v):
    M=v.reshape(chiwtemp,chiwtemp,chiytemp,chiytemp,chiytemp,chiytemp,chiwtemp,chiwtemp)
    Mscaled=rg_(M,gl,gr,gu,gnw,gne)
    return Mscaled.flatten()

n=chiwtemp**4*chiytemp**4

In [25]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  1.82548082,  1.84913751,  1.90390264,  1.90861535,
        2.04672911,  2.05618218,  2.79575673,  2.88839787,  3.2273013 ,
        3.29335963,  3.34877207,  3.37503155,  3.55961591,  3.67312588,
        3.74014019,  3.74075378,  3.74605071,  3.81223776,  3.84210499])

In [228]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


wgauged=w@u
# wgauged=w

def scaleop(v):
    M=v.reshape(chiw,chiw,chiw,chiw)
    mera=np.einsum(M,[0,1,2,3],U,[0,1,8,9],wgauged,[12,8,4],wgauged,[9,13,5],U.conj(),[2,3,10,11],wgauged.conj(),[12,10,6],wgauged.conj(),[11,13,7],optimize=("greedy",2**100))
    return mera.flatten()

n=chiw**4

import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=20,which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.18126637,  0.56473003,  0.7651075 ,  0.82930577,
        0.93717961,  1.15840441,  1.16340123,  1.21039609,  1.23688751,
        1.46377128,  1.54051149,  1.57518613,  1.57518613,  1.64407593,
        1.64407593,  1.64840365,  1.86336301,  1.8636821 ,  1.86368214])

In [11]:
# compress legs

# def wrenv_(A):
#     return np.einsum(A,[5,6,0,8],A,[4,7,1,6],A.conj(),[4,7,3,9],A.conj(),[5,9,2,8])
# def wlenv_(A):
#     return np.einsum(A,[0,6,5,8],A,[1,7,4,6],A.conj(),[3,7,4,9],A.conj(),[2,9,5,8])
# def wvenv_(A):
#     return np.einsum(A,[8,0,6,5],A,[6,1,7,4],A.conj(),[8,2,9,5],A.conj(),[9,3,7,4])
# chik=20
# wrenv=wrenv_(A)
# _,wr=compress(wrenv,chik)
# wlenv=wlenv_(A)
# _,wl=compress(wlenv,chik)
# wvenv=wvenv_(A)
# _,wv=compress(wvenv,chik)

# wrenv=wrenv_(Aold)
# _,wrold=compress(wrenv,chik)
# wlenv=wlenv_(Aold)
# _,wlold=compress(wlenv,chik)
# wvenv=wvenv_(Aold)
# _,wvold=compress(wvenv,chik)

# glw=np.einsum(gl,[3,4,1,2],wrold.conj(),[3,4,0])
# grw=np.einsum(gr,[3,4,1,2],wlold.conj(),[3,4,0])
# guw=np.einsum(gu,[3,4,1,2],wvold.conj(),[3,4,0])

# def scaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten()
# def rscaleop(v):
#     v4=v.reshape(chik,chik,chik,chik)
#     out=np.einsum(v4.conj(),[0,1,2,3],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wl,[4,5,0],wr,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3],optimize=('greedy',2**100))
#     return out.flatten().conj()

# n=chik**4

In [29]:
def Az_(Ainit,chi):
    #input: Ainit, chi, op: operator acting on a leg
    #output:up-down symmetric Aout
    chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

   #vertical cut
    w1,vtemp=compress(Ainit,chiw)
    v=vtemp.conj()*np.sqrt(w1)

    #horizontal cut
    vl2,w2,vr2,tr2= h_cut(Ainit,chiy)
    
    Aout=np.einsum(vr2*np.sqrt(w2),[4,5,0],v,[5,7,1],vl2*np.sqrt(w2),[6,7,2],v.conj(),[4,6,3])
    Anorm=np.linalg.norm(Aout)
    
    Azout=0.5*(np.einsum(vr2*np.sqrt(w2),[4,5,0],v,[5,7,1],sz@vl2*np.sqrt(w2),[6,7,2],v.conj(),[4,6,3])+np.einsum(vr2*np.sqrt(w2),[4,5,0],v,[5,7,1],vl2*np.sqrt(w2),[6,7,2],sz@v.conj(),[4,6,3]))
    Anorm=np.linalg.norm(Aout)
    return Azout/Anorm

def MOq_(A0,Az):
    return 0.5*(np.einsum(A0,[2,9,8,0],A0,[8,10,3,1],A0,[4,6,11,9],Az,[11,7,5,10])+np.einsum(A0,[2,9,8,0],Az,[8,10,3,1],A0,[4,6,11,9],A0,[11,7,5,10]))

vertical truncation err: 0.0
horizontal truncation err: 0.0
Aout shape: (4, 4, 4, 4)


In [31]:
MOq.shape

(4, 4, 4, 4, 4, 4, 4, 4)